<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />

# Generando música<a id="top"></a>

<i>Última actualización: 2025-03-05</small></i></div>
***

## Introducción

En este _notebook_ vamos a crear un modelo que aprenderá a «tocar música». El entrecomillado es porque, para tocar música bien, se necesitan modelos y técnicas muy complejas que quedan un poco fuera del alcance de un tutorial como este.

Sin embargo, en este ejercicio tocaremos los fundamentos de la generación basada en notas y acordes y, junto con los modelos que veremos más adelante en este tema más conceptos como redes bidireccionales e incrustaciones de la parte de procesamiento del lenguaje natural podremos generar música con un poco más de sentido.

## Objetivos

Crearemos un modelo de predicción de notas basado en una secuencia de notas anteriores. Al final habremos aprendido a:

- Leer y escribir ficheros midi,
- Generar secuencias siguiendo una tipología de red recurrente _one-to-many_, y
- Guardar modelos entrenados en disco para entrenarlos en diferentes momentos.

## Bibliotecas y configuración

A continuación importaremos las librerías que se utilizarán a lo largo del _notebook_.

In [ ]:
import glob
import os

import matplotlib.pyplot as plt
import music21
import numpy as np
import pandas as pd
import torch
import torchmetrics

import utils

También vamos a configurar algunos parámetros para adaptar la presentación grafica.

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

Crearemos los directorios necesarios para almacenar los ficheros de datos y los modelos generados.

In [ ]:
os.makedirs('tmp', exist_ok=True)

Y terminamos con constantes que se utilizarán a lo largo del _notebook_.

In [ ]:
BATCH_SIZE = 1024
TRAIN_EPOCHS = 2
SEQUENCE_LEN = 20

***

## Carga y preparación de datos

Empezaremos cargando todas las notas de los archivos `.mid` ubicados en la ruta relativa `datasets/Music`. Estas notas se almacenarán en una lista llamada `notes`. Algunos detalles de implementación:

- Para parsear un fichero midi utilizaremos la función `parse(file)` del módulo `converter` de la librería `music21`,
- Las notas que queremos obtener están en el atributo `.flat.notes` del midi analizado. Sin embargo, tenemos que tener en cuenta que contiene dos tipos de datos:
  - Notas normales, que son del tipo `note.Note`. Si la nota es `note`, almacenaremos directamente en la lista de notas la representación en cadena de `note.pitch`.
  - Acordes, que son del tipo `chord.Chord`. Son una lista de notas, y lo que almacenaremos será la lista de sus notas (si el acorde es `chord`, la lista será `chord.normalOrder`) como una cadena de texto donde cada nota irá separada por un punto (`'.'`).

Esto no es por capricho; es una forma de representar las notas que facilitará la conversión posterior de las notas generadas en una nueva pista de audio.

In [ ]:
midi_files = glob.glob("../Datasets/music/doom/*.mid")

notes = []  # Almacenará todas las notas y acordes de los ficheros
for file in midi_files:
    print(f'Parsing {file}')

    midi = music21.converter.parse(file)
    
    for note_or_chord in midi.flat.notes:
        if isinstance(note_or_chord, music21.note.Note):
            notes.append(str(note_or_chord.pitch))
        elif isinstance(note_or_chord, music21.chord.Chord):
            notes.append('.'.join(str(n) for n in note_or_chord.normalOrder))
    
    notes.append('EOS')  # Añadimos un token de fin de canción

print(f"Some notes: {notes[:10]}")

Nuestro siguiente paso será crear dos diccionarios: `note_to_int` y `int_to_note`. ¿Cuál será su significado? Pues que tenemos la lista de todas las notas de todas las pistas de sonido. Como las redes trabajan con números, no con símbolos, lo que haremos será asignar un valor único a cada nota diferente. Con estos dos diccionarios sabremos traducir de nota a número y de un número a su nota para poder traducir dentro y fuera de la red.

Por lo tanto, crearemos estos dos diccionarios con las diferentes notas en orden ascendente, desde 0 hasta el número de notas menos una.

In [ ]:
unique_notes = sorted(set(notes))
note_to_int = {note: i for i, note in enumerate(unique_notes)}
int_to_note = {i: note for note, i in note_to_int.items()}

print(f"Unique notes: {len(unique_notes)}")

Continuamos con la preparación de los conjuntos de datos. En `notas` tenemos la lista ordenada de notas. Se espera que las notas estén determinadas por la secuencia anterior. Por facilitar la implementación, no hemos creado un nuevo token para indicar que una canción ha terminado, por lo que habrá ciertas pausas que no se correspondan con un compás real. Si te apetece modificarlo, ¡adelante!

Lo que crearemos ahora será el conjunto de entrenamiento, las variables `x_train` y `y_train`. `y_train` estará formado por las secuencias de entrada, que tendrán una longitud de 50 (la variable SEQUENCE_LEN, ya creada), mientras que `y_train` tendrá la nota correspondiente a continuación de esa secuencia. Construiremos este conjunto a partir de la lista `notas`.

In [ ]:
current_song = []
x_data, y_data = [], []
for note in notes:
    current_song.append(note)
    if note == "EOS":  # Fin de canción, así que procesamos la secuencia
        if len(current_song) > SEQUENCE_LEN:  # Si hay suficientes notas
            for i in range(SEQUENCE_LEN, len(current_song)):
                seq_in = current_song[i - SEQUENCE_LEN:i]
                seq_out = current_song[i]
                x_data.append([note_to_int[n] for n in seq_in])
                y_data.append(note_to_int[seq_out])
        current_song = []

x_data = np.array(x_data)
y_data = np.array(y_data)

for i in range(10):
    print(f'{x_data[i]} -> {y_data[i]}')

print(f"X data shape: {x_data.shape}")
print(f"Y data shape: {y_data.shape}")

Normalizaremos los valores de entrada para que estén en el rango [0, 1]. Para ello, dividiremos entre el número de notas diferentes. Ojo, tendría más sentido usar la aproximación de usar _embeddings_ para representar las notas en lugar de usar el índice como tal, pero como todavía no sabemos qué es esto, nos conformamos con normalizarlos.

In [ ]:
x_data = x_data / len(unique_notes)

Como vamos a alimentar una red recurrente, las dimensiones de entrada deben ser $M \times T \times C$, siendo:

- $M$: El número de ejemplos, es decir, el número total de secuencias.
- $T$: El tamaño de la secuencia, que hemos definido en la constante anterior.
- $C$: El número de parámetros que tiene cada elemento de la secuencia, que en nuestro caso es $1$ (cada nota es un único entero).

El problema es que las dimensiones de nuestro conjunto de entrenamiento no son esas, por lo que tendremos que remodelar el conjunto de entrenamiento.

In [ ]:
x_data = np.expand_dims(x_data, axis=2)

print(f'X data shape: {x_data.shape}')

Por último crearemos un Dataset para este conjunto de datos y su `DataLoader` correspondiente.

In [ ]:
class MusicDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return (
            torch.tensor(self.x[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.long),
        )

dataset = MusicDataset(x_data, y_data)
data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

Y con los conjuntos creados y listos para entrenar, pasamos a trabajar con el modelo.

## Implementando y entrenando el modelo

Ahora crearemos un modelo para que aprenda a predecir la siguiente nota dada una secuencia de notas.

In [ ]:
class MusicGeneratorModel(torch.nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_classes,
        num_layers,
        dropout=0.2,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.rnn_layers = torch.nn.ModuleList()
        self.dpo_layers = torch.nn.ModuleList()

        for i in range(num_layers):
            in_size = input_size if i == 0 else hidden_size
            self.rnn_layers.append(
                torch.nn.GRU(in_size, hidden_size, batch_first=True)
            )
            self.dpo_layers.append(
                torch.nn.Dropout(dropout)
            )
        
        self.fc = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        for gru, dropout in zip(self.rnn_layers, self.dpo_layers):
            x, _ = gru(x)
            x = dropout(x)
        x = x[:, -1, :]
        logits = self.fc(x)
        return logits

music_generator = MusicGeneratorModel(
    input_size=1,
    hidden_size=128,
    num_classes=len(unique_notes),
    num_layers=3,
    dropout=0.2,
)

Ahora entrenaremos el modelo con nuestro conjunto de datos durante 25 épocas; no son muchas, pero la carga computacional derivada del entrenamiento de este tipo de modelos es bastante pesada.

In [ ]:
history = utils.train(
    model=music_generator,
    train_loader=data_loader,
    n_epochs=TRAIN_EPOCHS,
    criterion=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(music_generator.parameters()),
    validation_split=0.1,
    metric_fn=torchmetrics.classification.MulticlassAccuracy(num_classes=len(unique_notes)),
)

Antes de ver la evolución del error, podríamos modificar la creación del modelo para que cargue el mejor punto de control si existe y si queremos (mediante una variable, por ejemplo `LOAD_PREVIOUS`).

Ahora, veamos cómo han evolucionado error y exactitud.

In [ ]:
pd.DataFrame(history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

## A generar música

Ya tenemos un modelo entrenado para generar música. Ahora procederemos a generar una canción. Para hacerlo sencillo, generaremos una canción de N notas (digamos 100, y veremos cómo se comporta. Para ello haremos lo siguiente

1. Crear una secuencia aleatoria de inicio del tamaño de secuencia esperado, lo que constituirá nuestra primera entrada,
2. Pasar esa secuencia al modelo y recoger la siguiente nota que predice,
3. Eliminar la primera nota de la secuencia y añadir la nueva nota al final, lo que constituirá la siguiente secuencia, y
4. Continuar así hasta terminar de generar notas.

El resultado será una lista con la secuencia y todas las notas generadas. La lista con las notas generadas se llamará `new_song`.

In [ ]:
start_index = eos_index = note_to_int["EOS"]
while start_index == eos_index:
    start_index = np.random.randint(0, len(dataset))

pattern, _ = dataset[start_index]
pattern = pattern.unsqueeze(0)  # (SEQUENCE_LEN, 1) -> (1, SEQUENCE_LEN, 1)
pattern.shape

generated_song = []
max_generated = 256

music_generator.eval()
for _ in range(max_generated):
    with torch.no_grad():
        output = music_generator(pattern)
        probabilities = torch.softmax(output, dim=1).cpu().numpy().flatten()
        next_index = np.random.choice(len(probabilities), p=probabilities)
    
    if next_index == eos_index:
        break
    
    generated_song.append(int_to_note[next_index])
    
    next_value = next_index / float(len(unique_notes))
    pattern_np = pattern.cpu().numpy()
    pattern_np = np.roll(pattern_np, -1, axis=1)
    pattern_np[0, -1, 0] = next_value
    pattern = torch.tensor(pattern_np)

print(f"Generated song: {generated_song}")

El siguiente fragmento de código transforma la lista de notas en un midi, separando cada nota por medio segundo.

In [ ]:
offset = 0
output_notes = []

for pattern in generated_song:
    if ('.' in pattern) or pattern.isdigit():
        # Acorde, así que lo partimos en sus notas
        notes = []
        for current_note in pattern.split('.'):
            new_note = music21.note.Note(int(current_note))
            new_note.storedInstrument = music21.instrument.Violin()
            notes.append(new_note)
        new_chord = music21.chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        # Nota, así que la creamos directamente
        new_note = music21.note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = music21.instrument.Violin()
        output_notes.append(new_note)

    offset += 0.5

midi_stream = music21.stream.Stream(output_notes)
midi_stream.write('midi', fp='tmp/test_output.mid')

Como puedes ver, el modelo ha generado una canción. Sí, no respetamos los tiempos, hay secuencias que no tienen sentido (los cortes entre canciones), etc., pero nos ha servido como experimento para ver el desarrollo de un proyecto de principio a fin.

## Conclusiones

Hemos implementado un modelo recurrente que aprende de muchos datos para resolver un problema _de_uno_a_muchos_: generar música a partir de una semilla inicial.

Os animamos a modificar la arquitectura para ver si encontráis una que genere canciones que tengan algún sentido, y a probar a añadir entradas aleatorias durante el entrenamiento para que durante la inferencia se puedan añadir dichas entradas para alterar la generación de melodías.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>